In [15]:
import pandas as pd

import spacy
from scispacy.abbreviation import AbbreviationDetector

from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

import html
import emoji
from googletrans import Translator

from pathlib import Path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from database import database_manager as dbm

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/raimuu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


**Extract**

In [16]:
%%script true
# Skip this cell because old code for extracting data
read in raw data
file_path = "../../data/raw/covidTwitterData.zip"
df = pd.read_csv(file_path, compression='zip')


In [17]:

'''
Query Usage:
table = "raw_twitter_data" OR
table = "processed_twitter_data"

Refer to alembic migration versions for the full schema
'''

table = "raw_twitter_data"
query = f"""
    SELECT * 
    FROM "{table}" 
"""
df = dbm.query_db(query)
display(df)

# df[['text', 'date']].to_csv("../../data/processed/output.txt", index=False, header=False)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Ⓜ️ark Holton,None,Principal Engineer #Java #Kafka #Golang. 26.2 ...,2007-07-08 19:07:00,564.0,497.0,17529.0,False,2020-11-09 17:38:55,Great news from @pfizer on #COVID19 vaccine ef...,"['COVID19', 'CovidVaccine']",Twitter for Android,False
1,Emily Blake,Norwich,It does not do to dwell on dreams and forget t...,2011-04-07 18:46:40,167.0,305.0,1298.0,False,2020-11-09 17:38:52,Also. I get the elderly should get vaccinated ...,None,Twitter for iPhone,False
2,Comrade McBot,"Glasgow, Scotland",Grievance chimp. 🇪🇺🦄,2012-03-18 14:35:26,1734.0,1267.0,145702.0,False,2020-11-09 17:38:51,The Brigadier looks a bit traumatised being in...,None,Twitter for Android,False
3,Doug P,"J10i2 Epsom, United Kingdom",Hello! Hope you like my sheep! Back on here to...,2010-01-12 18:57:31,581.0,795.0,24647.0,False,2020-11-09 17:38:48,Keep going folks. Slowly but surely we are wi...,['CovidVaccine'],Twitter for Android,False
4,LizzDregne,"Cleveland, OH","Happy Wife, Proud Mother & Hard-Working QA Ana...",2014-03-15 01:11:58,842.0,1170.0,27594.0,False,2020-11-09 17:38:41,"@GeraldoRivera @realDonaldTrump Animal Farm, C...",None,Twitter for iPhone,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399640,troovus,Conquest Arriving Coherent,The #HousingCrisis is cultivated to widen ineq...,2013-04-28 13:03:04,6963.0,5378.0,16057.0,False,2020-11-09 17:39:21,When you see the light up at the end of the tu...,None,Twitter for Android,False
399641,debi boeras @GHIG,None,Virologist. Molecular biologist working on inf...,2015-05-11 14:46:54,149.0,305.0,620.0,False,2020-11-09 17:39:19,We need tremendous innovation in supply chain....,['CovidVaccine'],Twitter for iPhone,False
399642,Chris Beaumont,None,None,2011-06-01 18:04:19,292.0,525.0,8548.0,False,2020-11-09 17:39:05,When your stood at Bromborough and you can see...,"['COVID19', 'CovidVaccine']",Twitter Web App,False
399643,Kelly,None,"Wife, Mother, Travel lover.",2012-08-29 17:08:00,133.0,717.0,2727.0,False,2020-11-09 17:39:00,Teachers should be priority also!! #CovidVacci...,"['CovidVaccine', 'COVID19']",Twitter for Android,False


**Transform**

General Cleaning

In [18]:
def unescape_html(text): 
    return html.unescape(text)

df_clean = df.dropna().copy() # drop all rows containing null

df_clean = df_clean[~df_clean['text'].str.contains(r'\… https://t\.co/\S+', regex=True)] # remove tweets with this pattern from twitter V2 migration

df_clean['text'] = (
    df_clean['text']
    .str.replace(r'http\S+|www.\S+', '', regex=True) # remove links
    .str.replace(r'[\n\t]', ' ', regex=True) # remove \t\n
    .apply(unescape_html) # escape html formatting
    .apply(emoji.demojize) # replace emojis with their symbolic name
    .str.replace(r'[^\w\s.,!?;:\-()\'"/&]', '', regex=True) # remove non-alphanumeric characters (symbols not in this set)
    .str.replace(":", " ")
    .str.replace("_", " ")
)

df_clean = df_clean.drop_duplicates(subset='text') # remove duplicates after cleaning/unifying format

# convert columns to appropriate data types
df_clean['user_friends'] = pd.to_numeric(df_clean['user_friends']).replace({False: 0, True: 1}).astype(int)
df_clean['user_followers'] = pd.to_numeric(df_clean['user_followers']).astype(int)
df_clean['user_favourites'] = pd.to_numeric(df_clean['user_favourites']).astype(int)
df_clean['date'] = pd.to_datetime(df_clean['date']).dt.date
df_clean['user_created'] = pd.to_datetime(df_clean['user_created']).dt.date
display(df_clean)

# df_clean[['text', 'date']].to_csv("../../data/processed/output_clean.txt", index=False, header=False)



,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
3,Doug P,"J10i2 Epsom, United Kingdom",Hello! Hope you like my sheep! Back on here to...,2010-01-12,581,795,24647,False,2020-11-09,Keep going folks. Slowly but surely we are wi...,['CovidVaccine'],Twitter for Android,False
6,𝓐𝓵𝓮𝔁,UK,"Luminary by day. Wine aficionado, astronomer &...",2010-02-04,796,692,37237,False,2020-11-09,I definitely wont be getting the CovidVaccine ...,['CovidVaccine'],Twitter for iPhone,False
7,Craig Justice,winchester,Huge #saintsfc fan also a Huge NFL fan too. Ch...,2012-08-29,368,254,2007,False,2020-11-09,In other words upside-down face upside-down f...,"['vaccine', 'CovidVaccine', 'Pressconference',...",Twitter for iPhone,False
8,Dr. Sarah Kelly,"England, United Kingdom","PhD in Ant Behaviour 🐜(really!), writes thrill...",2014-02-18,297,475,9486,False,2020-11-09,"Jeez! Trains, bugles, football - whats next?? ...","['CovidVaccine', 'BorisJohnson']",Twitter for iPhone,False
11,Ludo Bok,New York,"Team Leader @UNDP working on #HIV, #health and...",2011-09-13,46374,27427,875,False,2020-11-09,Pfizers Early Data Shows Vaccine Is More Than ...,['CovidVaccine'],Twitter for iPhone,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399627,Dr Iain MacInnes,"Dornoch, Scotland",Senior Lecturer in #medieval #military #Scotla...,2016-05-10,1924,2131,98499,False,2020-11-09,"Wow, there are some pretty tortured similes be...","['briefing', 'BorisJohnson', 'PrimeMinister', ...",Twitter Web App,False
399628,ĐuryoĐan ©,ഹസ്തിനപുരം,HINDU 🕉 Indian 🇮🇳 Proud Hardcore Nationalist ....,2010-05-11,4067,2817,117712,False,2020-11-09,Pfizer is the first one to claim a successful ...,['CovidVaccine'],Twitter for iPhone,False
399634,Human Resource Executive,"Palm Beach Gardens, FL",The premier publication focused on strategic i...,2010-11-22,15853,1324,4768,False,2020-11-09,What do HR leaders need to know now about COVI...,"['HR', 'leaders', 'COVID19', 'vaccine', 'welln...",Hootsuite Inc.,False
399638,Chris Taps,"UK, London.",Movie & TV fan who appreciates a good pub and ...,2012-09-22,152,384,1341,False,2020-11-09,Van Tam bringing a train metaphor to the table...,"['COVID19', 'CovidVaccine', 'Pressconference']",Twitter for iPhone,False


Abbreviation Expansion

In [19]:
%%script true

med_abrv_file_path = Path("../../data/abbreviations/med_abbreviations.txt")
df_abrv = pd.read_csv(med_abrv_file_path)
abrv_dict = dict(zip(df_abrv['Abbreviation'], df_abrv['Full Form']))

# if error, run: python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("abbreviation_detector") # select the model

translator = Translator()

# expands abbreviations using spacy to inference definitions 
def abbreviation_expander(text):
    doc = nlp(text) # pass the text into the model
    for abrv in doc._.abbreviations:
        # text = text.replace(f"({abrv.text})", "")
        text = text.replace(abrv.text, abrv._.long_form.text)
    return text

# expands abbreviated medical terms defined in a text file
def med_abbreviation_expander(text):
    words = text.split()
    res = [abrv_dict[word] if word in abrv_dict else word for word in words] # replace the abbreviation with full form
    return " ".join(res) # return the full text without abbreviations 

# keeps abbreviations but makes the rest lowercase
def lower_except_abbreviation(text):
    words = text.split()
    res = [word if word in abrv_dict else word.lower() for word in words] # replace the abbreviation with full form
    return " ".join(res) # return the full text without abbreviations 

# translates text
def translate_text(text):
    translation = translator.translate(text, dest='en')
    return translation.text
    
df_clean['text'] = (
    df_clean['text']
    .apply(abbreviation_expander)
)

# df_clean[['text', 'date']].to_csv('../../data/processed/output_abr_expand.txt', index=False, header=False)


**Classification** using VADER

In [20]:
analyzer = SentimentIntensityAnalyzer()
sentiment_scores = []

for _, row in df_clean.iterrows():
    sentiment = analyzer.polarity_scores(row['text'])
    sentiment['tweet'] = row['text']
    sentiment['hashtag'] = row['hashtags']
    sentiment_scores.append(sentiment)

df_sentiments = pd.DataFrame(sentiment_scores)
display(df_sentiments)

,neg,neu,pos,compound,tweet,hashtag
0,0.000,0.478,0.522,0.8671,Keep going folks. Slowly but surely we are wi...,['CovidVaccine']
1,0.093,0.705,0.202,0.3565,I definitely wont be getting the CovidVaccine ...,['CovidVaccine']
2,0.000,1.000,0.000,0.0000,In other words upside-down face upside-down f...,"['vaccine', 'CovidVaccine', 'Pressconference',..."
3,0.109,0.634,0.257,0.5502,"Jeez! Trains, bugles, football - whats next?? ...","['CovidVaccine', 'BorisJohnson']"
4,0.000,0.748,0.252,0.5209,Pfizers Early Data Shows Vaccine Is More Than ...,['CovidVaccine']
...,...,...,...,...,...,...
188701,0.146,0.569,0.285,0.5267,"Wow, there are some pretty tortured similes be...","['briefing', 'BorisJohnson', 'PrimeMinister', ..."
188702,0.000,0.595,0.405,0.8176,Pfizer is the first one to claim a successful ...,['CovidVaccine']
188703,0.000,0.838,0.162,0.4404,What do HR leaders need to know now about COVI...,"['HR', 'leaders', 'COVID19', 'vaccine', 'welln..."
188704,0.000,0.752,0.248,0.6467,Van Tam bringing a train metaphor to the table...,"['COVID19', 'CovidVaccine', 'Pressconference']"


**Load**

In [21]:
%%script true
# file_path = "../../data/processed/cleanCovidTwitterData.zip"
# df_head = df_clean.head()
# df_head.to_csv(file_path, compression='zip', index=False)

"""
Create a new table with the processed data
    dbm.create_table("table_name", dataframe)
"""

"""
Replace a table with the processed data
    dbm.create_table("table_name", dataframe, replace=True)
"""
newTable = "processed_twitter_data"
dbm.create_table(table_name=newTable, dataframe=df_clean, replace=True)

query = f"""
    SELECT * FROM "{newTable}"
"""
df = dbm.query_db(query)
display(df)





In [ ]:
%%script true
newTable = "sentiment_scores"
dbm.create_table(table_name=newTable, dataframe=df_sentiments, replace=True)

query = f"""
    SELECT * FROM "{newTable}"
"""
df = dbm.query_db(query)
display(df)

The table 'sentiment_scores' already exists. Replacing entries.


,neg,neu,pos,compound,tweet,hashtag
0,0.000,0.478,0.522,0.8671,Keep going folks. Slowly but surely we are wi...,['CovidVaccine']
1,0.093,0.705,0.202,0.3565,I definitely wont be getting the CovidVaccine ...,['CovidVaccine']
2,0.000,1.000,0.000,0.0000,In other words upside-down face upside-down f...,"['vaccine', 'CovidVaccine', 'Pressconference',..."
3,0.109,0.634,0.257,0.5502,"Jeez! Trains, bugles, football - whats next?? ...","['CovidVaccine', 'BorisJohnson']"
4,0.000,0.748,0.252,0.5209,Pfizers Early Data Shows Vaccine Is More Than ...,['CovidVaccine']
...,...,...,...,...,...,...
188701,0.146,0.569,0.285,0.5267,"Wow, there are some pretty tortured similes be...","['briefing', 'BorisJohnson', 'PrimeMinister', ..."
188702,0.000,0.595,0.405,0.8176,Pfizer is the first one to claim a successful ...,['CovidVaccine']
188703,0.000,0.838,0.162,0.4404,What do HR leaders need to know now about COVI...,"['HR', 'leaders', 'COVID19', 'vaccine', 'welln..."
188704,0.000,0.752,0.248,0.6467,Van Tam bringing a train metaphor to the table...,"['COVID19', 'CovidVaccine', 'Pressconference']"
